# Finite State, Discrete Time Markov Chains
---

In [2]:
from header import *

## Overview

There are 3 different methods of finding $\vec{\pi}$ (which represents the probability of being in each state on average):
1. evolution via sample evolution
2. evolution via matrix multiplication
3. evolution via eigenvalues and eigenvectors

We are going to write methods for each and see how their results compare.

## System Checks

First let's write a method to do some basic checks:
1. Check to make sure the dimensions of the transition matrix and distribution matrix are compatible
2. Each row of the transition matrix sums to 1

In [3]:
def architecture_check_passed(transition_matrix, initial_distribution_matrix):
    num_of_states = transition_matrix.shape[0]
    if initial_distribution_matrix.shape[0] != num_of_states:
        #print("Transition matrix and initial distribution dimensions don't match")
        return False

    row_sums  = np.sum(transition_matrix, 1)
    should_be = np.ones_like(row_sums)
    if np.allclose(row_sums, should_be) == False:
        #print('Rows of transition matrix do not all sum to 1')
        return False
    return True

## Sample Evolution

1. Create a matrix called "simulation_ledger" and let it evolve through random choices
2. Compute empirical probability distribution function - this is done by simulating the behavior of samples

In [4]:
def sample_evolution(initial_parameters):
    # unpack list
    num_of_steps                = initial_parameters[0]
    num_of_records              = initial_parameters[1]
    num_of_states               = initial_parameters[2]
    num_of_samples              = initial_parameters[3]
    transition_matrix           = initial_parameters[4]
    initial_distribution_matrix = initial_parameters[5]
    
    ### -------------- set stage for simulation -------------- ###
    # create empirical probability distribution function
    epdf                               = np.zeros([num_of_records, num_of_states], dtype = float)
    # initial the epdf by writing the initial distribution matrix to epdf
    epdf[0]                            = initial_distribution_matrix[:]
    #printltx(r"initialized epdf = " + ltxmtx(epdf))
    
    scaled_initial_distribution_matrix = np.rint(initial_distribution_matrix*num_of_samples).astype(int)
    simulation_ledger                  = np.zeros([num_of_records, num_of_samples], dtype = int)

    # initialize simulation ledger by distributing probabilities
    # from the initial distribution matrix to the sample population
    simulation_ledger_index_1 = 0
    for state_index in range(num_of_states):
        simulation_ledger_index_2 = simulation_ledger_index_1 + scaled_initial_distribution_matrix[state_index]
        simulation_ledger[0, simulation_ledger_index_1:simulation_ledger_index_2] = state_index
        simulation_ledger_index_1 = simulation_ledger_index_2
    # shuffle the columns in the first row
    np.random.shuffle(simulation_ledger[0])
    
    ### -------------- run the simulation -------------- ###
    for step in range(num_of_steps):
        # "step" here refers to the location with respect to the number of records
        # since we're looping through the same array
        current_step = step % num_of_records
        next_step    = (step + 1) % num_of_records
        for sample in range(num_of_samples):
            current_state      = simulation_ledger[current_step, sample]
            # choose random number between 0 and 1
            random_probability = np.random.rand()
            # randomly decide which state sample goes to next
            for next_state in range(num_of_states):
                random_probability -= transition_matrix[current_state, next_state]
                if random_probability < 0:
                    simulation_ledger[next_step, sample] = next_state
                    break
        epdf[next_step, :] = np.histogram(simulation_ledger[next_step, :], \
                                          normed = True, bins = range(num_of_states + 1))[0]
    
    # right now, the last distribution is not necessarily on the bottom of the epdf matrix;
    # so, the following code rearranges the rows of the matrix to make sure the final distribution is on bottom
    epdf = np.roll(epdf, num_of_records - next_step - 1, axis = 0)
    return epdf

## Matrix Multiplication Evolution

Evolution via matrix multiplication is the mathy way to achieve similar results to doing the sample evolution above. Below are some things to keep in mind:

1. p(i, j) = probability that a sample at state i now will jump to state j next
2. the arrows (if you're looking at a diagram of the random walk) out of each state better add up to 1
    * sum(transition_matrix[i, :]) = 1 where row is where you are now and column where you might go next
3. you can encode the probability of going from one state to another in a transition matrix $TransitionMatrix_{i j}$

In [5]:
def matrix_multiplication_evolution(initial_parameters):
    # unpack list
    num_of_steps                = initial_parameters[0]
    num_of_records              = initial_parameters[1]
    num_of_states               = initial_parameters[2]
    transition_matrix           = initial_parameters[3]
    initial_distribution_matrix = initial_parameters[4]
    
    ### -------------- set stage for simulation -------------- ###
    # create ________ probability distribution function
    tpdf    = np.zeros([num_of_records, num_of_states], dtype = float)
    # initialize the tpdf by writing the initial distribution matrix to tpdf
    tpdf[0] = initial_distribution_matrix[:]
    printltx(r"initialized tpdf = " + ltxmtx(tpdf))
    
    ### -------------- run the simulation -------------- ###
    for step in range(num_of_steps):
        current_step       = step % num_of_records
        next_step          = (step + 1) % num_of_records
        tpdf[next_step, :] = tpdf[current_step, :].dot(transition_matrix)
    
    # right now, the last distribution is not necessarily on the bottom of the epdf matrix;
    # so, the following code rearranges the rows of the matrix to make sure the final distribution is on bottom
    tpdf = np.roll(tpdf, num_of_records - next_step - 1, axis = 0)
    
    return tpdf

## Eigenvalues/Eigenvectors Evolution

Computing future behavior in this way actually doesn't even depend on the present.

The transition matrix will be $P = L^{-1}JL$ where $L$ has rows that are left eigenvectors of $P$.  
$J$ is a diagonal matrix with each element on its diagonal being a Jordan block.

In [6]:
def normalize_rows(left_eigenvectors):
    tol = 0.001
    n = left_eigenvectors.shape[0]
    for i in range(n):
        s = np.sum(left_eigenvectors[i, :])
        if abs(s) > tol:
            left_eigenvectors[i, :] *= 1/s
        else:
            for j in range(n):
                if abs(left_eigenvectors[i, j]) > tol:
                    left_eigenvectors[i, :] *= 1/left_eigenvectors[i, j]
                    break
    return left_eigenvectors

def eigen_future(transition_matrix, initial_distribution_matrix):
    # numpy gives us Pv = lambda*v which return right eigenvectors
    # but we want vP = lambda*v
    (eigenvalues, left_eigenvectors_transpose) = np.linalg.eig(transition_matrix.T) # returns left vectors but they're in columns
    left_eigenvectors                          = left_eigenvectors_transpose.T
    
    # sort so that largest eigenvalue is first (descending length, then descending real part,
    # then descending imaginary part)
    eigenvalue_lengths                         = np.abs(eigenvalues).round(4) # must round, else eigenvalues of same true length may appear to be of different lengths
    eigenvalue_real_part                       = np.real(eigenvalues).round(4)
    eigenvalue_imaginary_part                  = np.imag(eigenvalues).round(4)
    sort                                       = np.lexsort([eigenvalue_imaginary_part, eigenvalue_real_part, eigenvalue_lengths]) # sort priority goes right to left, for some reason
    # lexsort does ascending sort; this changes to descending, as desired
    sort                                       = sort[::-1]
    eigenvalues                                = eigenvalues[sort]
    left_eigenvectors                          = left_eigenvectors[sort]
    eigenvalue_lengths                         = eigenvalue_lengths[sort]
    
    left_eigenvectors                          = normalize_rows(left_eigenvectors)
    right_eigenvectors                         = np.linalg.inv(left_eigenvectors)
    diagonal_matrix                            = np.diag(eigenvalues)
    
    num_of_states                              = transition_matrix.shape[0]
    transient_states                           = eigenvalue_lengths < 1
    period                                     = num_of_states - np.sum(transient_states)
    diagonal_matrix_copy                       = diagonal_matrix.copy()
    # we know all |lambda| < 1 die in the long run so we jus set them to 0 here
    diagonal_matrix_copy[:, transient_states]  = 0
    
    pdf = np.zeros([period, num_of_states])
    for step in range(period):
        pdf[step, :] = initial_distribution_matrix.dot(right_eigenvectors).dot(diagonal_matrix_copy**(step + 1)).dot(left_eigenvectors)
    printltx(r"The analysis via eigenvectors says that, in the long run, this sytem should oscillate through the distributions below.")
    display(pdf)
    printltx(r"The average of these distributions is " + ltxmtx(np.mean(pdf, 0)))
    return left_eigenvectors

### Conclusion Function

In [7]:
def print_conclusions(num_of_steps, num_of_records, num_of_samples, pdf, method):
    if method == 'sample_evolution':
        printltx(r"I did " + str(num_of_steps) + " steps with " + str(num_of_samples) + \
                 " samples. The last " + str(num_of_records) + \
                 " distributions are written as below:")
        display(pdf)
        printltx(r"The average of these distributions, $\vec{\pi}$, is " + ltxmtx(np.mean(pdf, 0)))
    if method == 'matrix_multiplication_evolution':
        printltx(r"I did " + str(num_of_steps) + " steps. The last " + \
                 str(num_of_records) + " distributions are written as below:")
        display(pdf)
        printltx(r"The average of these distributions, $\vec{\pi}$, is " + ltxmtx(np.mean(pdf, 0)))
    if method == 'eigen_future':
        return None

## Testing the Methods

Consider the biased, non-lazy random walk on $\{0, 1, 2, ..., N - 1\}$.  
At interior states $\{1, 2, ..., N - 2\}$, there is a probability $p$ of jumping right and $q = 1 - p$ of jumping left.

We're going to test the above 3 methods on this random walk using 3 different boundary conditions on the boundary states ($0$ and $N - 1$) of the walk:
1. absorbing - once you reach $0$ and $N - 1$, you stay there with probability $1$
2. reflecting - once you reach $0$ and $N - 1$, you next jump to the adjacent interior state with probability $1$
3. semi-reflecting - At $0$, you jump to $1$ with probability $p$ and stay at $0$ with probability $q$. At $N - 1$, you jump to $N - 2$ with probability $q$ and stay at $N - 1$ with probability $p$.

Each boundary condition will have a different transition matrix to describe it so it'd be best to create a function that will take any given transition matrix and apply a boundary condition on it.

In [8]:
def apply_boundary_condition(condition, transition_matrix, p):
    q = 1 - p
    num_of_states = transition_matrix[0].shape[0]
    if condition == 'absorbing':
        # create beginning boundary
        beginning_boundary    = np.zeros(num_of_states)
        beginning_boundary[0] = 1
        # create end boundary
        end_boundary          = np.zeros(num_of_states)
        end_boundary[-1]      = 1
        # apply boundaries to transition matrix
        transition_matrix[0]  = beginning_boundary
        transition_matrix[-1] = end_boundary
        return transition_matrix
    elif condition == 'reflecting':
        # create beginning boundary
        beginning_boundary    = np.zeros(num_of_states)
        beginning_boundary[1] = 1
        # create end boundary
        end_boundary          = np.zeros(num_of_states)
        end_boundary[-2]      = 1
        # apply boundaries to transition matrix
        transition_matrix[0]  = beginning_boundary
        transition_matrix[-1] = end_boundary
        return transition_matrix
    elif condition == 'semi-reflecting':
        # create beginning boundary
        beginning_boundary    = np.zeros(num_of_states)
        beginning_boundary[0] = q
        beginning_boundary[1] = p
        # create end boundary
        end_boundary          = np.zeros(num_of_states)
        end_boundary[-1]      = p
        end_boundary[-2]      = q
        # apply boundaries to transition matrix
        transition_matrix[0]  = beginning_boundary
        transition_matrix[-1] = end_boundary
        return transition_matrix
    else:
        return transition_matrix

** Initialize Values **

Define initial distribution matrix and other initial conditions.

In [9]:
initial_distribution_matrix = np.array([0, 0, 1, 0, 0])

num_of_steps   = 1000
num_of_records = 10
num_of_samples = 2000
num_of_states  = initial_distribution_matrix.shape[0]
p              = 0.6

Define a function to create a transition matrix based on $p$ and number of states.

In [10]:
def create_transition_matrix(p, num_of_states):
    """ Creates an nxn transition matrix for a biased non-lazy random walk"""
    q = 1 - p
    transition_matrix = np.zeros([num_of_states, num_of_states])
    for row in range(num_of_states):
        if row+1 != num_of_states-1:            
            transition_matrix[row+1, row]   = q
            transition_matrix[row+1, row+2] = p
        else:
            return transition_matrix

### Testing Sample Evolution

**Absorbing Test**

In [11]:
transition_matrix  = create_transition_matrix(p, num_of_states)
transition_matrix  = apply_boundary_condition('absorbing', transition_matrix, p)

initial_parameters = [num_of_steps, num_of_records, num_of_states, num_of_samples,
                     transition_matrix, initial_distribution_matrix]

# run the test
printltx(r"Transition matrix $=$" + ltxmtx(transition_matrix))
printltx(r"Initial distribution $=$" + ltxmtx(initial_distribution_matrix))
if architecture_check_passed:
    sample_evolution_epdf = sample_evolution(initial_parameters)
    print_conclusions(num_of_steps, num_of_records, num_of_samples, sample_evolution_epdf, 'sample_evolution')

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

array([[ 0.292,  0.   ,  0.   ,  0.   ,  0.708],
       [ 0.292,  0.   ,  0.   ,  0.   ,  0.708],
       [ 0.292,  0.   ,  0.   ,  0.   ,  0.708],
       [ 0.292,  0.   ,  0.   ,  0.   ,  0.708],
       [ 0.292,  0.   ,  0.   ,  0.   ,  0.708],
       [ 0.292,  0.   ,  0.   ,  0.   ,  0.708],
       [ 0.292,  0.   ,  0.   ,  0.   ,  0.708],
       [ 0.292,  0.   ,  0.   ,  0.   ,  0.708],
       [ 0.292,  0.   ,  0.   ,  0.   ,  0.708],
       [ 0.292,  0.   ,  0.   ,  0.   ,  0.708]])

<IPython.core.display.Latex object>

** Reflecting Test **

In [12]:
transition_matrix  = create_transition_matrix(p, num_of_states)
transition_matrix  = apply_boundary_condition('reflecting', transition_matrix, p)

initial_parameters = [num_of_steps, num_of_records, num_of_states, num_of_samples,
                     transition_matrix, initial_distribution_matrix]

# run the test
printltx(r"Transition matrix $=$" + ltxmtx(transition_matrix))
printltx(r"Initial distribution $=$" + ltxmtx(initial_distribution_matrix))
if architecture_check_passed:
    sample_evolution_epdf = sample_evolution(initial_parameters)
    print_conclusions(num_of_steps, num_of_records, num_of_samples, sample_evolution_epdf, 'sample_evolution')

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

array([[ 0.    ,  0.31  ,  0.    ,  0.69  ,  0.    ],
       [ 0.1335,  0.    ,  0.4455,  0.    ,  0.421 ],
       [ 0.    ,  0.3125,  0.    ,  0.6875,  0.    ],
       [ 0.13  ,  0.    ,  0.4515,  0.    ,  0.4185],
       [ 0.    ,  0.3115,  0.    ,  0.6885,  0.    ],
       [ 0.1225,  0.    ,  0.47  ,  0.    ,  0.4075],
       [ 0.    ,  0.3085,  0.    ,  0.6915,  0.    ],
       [ 0.127 ,  0.    ,  0.481 ,  0.    ,  0.392 ],
       [ 0.    ,  0.3155,  0.    ,  0.6845,  0.    ],
       [ 0.128 ,  0.    ,  0.4635,  0.    ,  0.4085]])

<IPython.core.display.Latex object>

** Semi-Reflecting Test **

In [13]:
transition_matrix  = create_transition_matrix(p, num_of_states)
transition_matrix  = apply_boundary_condition('semi-reflecting', transition_matrix, p)

initial_parameters = [num_of_steps, num_of_records, num_of_states, num_of_samples,
                     transition_matrix, initial_distribution_matrix]

# run the test
printltx(r"Transition matrix $=$" + ltxmtx(transition_matrix))
printltx(r"Initial distribution $=$" + ltxmtx(initial_distribution_matrix))
if architecture_check_passed:
    sample_evolution_epdf = sample_evolution(initial_parameters)
    print_conclusions(num_of_steps, num_of_records, num_of_samples, sample_evolution_epdf, 'sample_evolution')

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

array([[ 0.0895,  0.103 ,  0.179 ,  0.243 ,  0.3855],
       [ 0.0735,  0.119 ,  0.1615,  0.2745,  0.3715],
       [ 0.079 ,  0.108 ,  0.178 ,  0.2505,  0.3845],
       [ 0.072 ,  0.1215,  0.155 ,  0.259 ,  0.3925],
       [ 0.0775,  0.096 ,  0.1775,  0.258 ,  0.391 ],
       [ 0.07  ,  0.118 ,  0.163 ,  0.2535,  0.3955],
       [ 0.0835,  0.1015,  0.174 ,  0.256 ,  0.385 ],
       [ 0.072 ,  0.1145,  0.1635,  0.258 ,  0.392 ],
       [ 0.0655,  0.111 ,  0.171 ,  0.25  ,  0.4025],
       [ 0.0685,  0.114 ,  0.16  ,  0.2635,  0.394 ]])

<IPython.core.display.Latex object>

### Testing Matrix Multiplication Evolution

** Absorbing Test **

In [14]:
num_of_samples     = None

transition_matrix  = create_transition_matrix(p, num_of_states)
transition_matrix  = apply_boundary_condition('absorbing', transition_matrix, p)

initial_parameters = [num_of_steps, num_of_records, num_of_states,
                     transition_matrix, initial_distribution_matrix]

if architecture_check_passed:
    matrix_multiplication_evolution_tpdf = matrix_multiplication_evolution(initial_parameters)
    print_conclusions(num_of_steps, num_of_records, num_of_samples, matrix_multiplication_evolution_tpdf, 'matrix_multiplication_evolution')

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

array([[  3.07692308e-001,   6.55348775e-159,   0.00000000e+000,
          9.83023163e-159,   6.92307692e-001],
       [  3.07692308e-001,   0.00000000e+000,   7.86418530e-159,
          0.00000000e+000,   6.92307692e-001],
       [  3.07692308e-001,   3.14567412e-159,   0.00000000e+000,
          4.71851118e-159,   6.92307692e-001],
       [  3.07692308e-001,   0.00000000e+000,   3.77480895e-159,
          0.00000000e+000,   6.92307692e-001],
       [  3.07692308e-001,   1.50992358e-159,   0.00000000e+000,
          2.26488537e-159,   6.92307692e-001],
       [  3.07692308e-001,   0.00000000e+000,   1.81190829e-159,
          0.00000000e+000,   6.92307692e-001],
       [  3.07692308e-001,   7.24763318e-160,   0.00000000e+000,
          1.08714498e-159,   6.92307692e-001],
       [  3.07692308e-001,   0.00000000e+000,   8.69715981e-160,
          0.00000000e+000,   6.92307692e-001],
       [  3.07692308e-001,   3.47886392e-160,   0.00000000e+000,
          5.21829589e-160,   6.92307692

<IPython.core.display.Latex object>

** Reflecting Test **

In [15]:
num_of_samples     = None

transition_matrix  = create_transition_matrix(p, num_of_states)
transition_matrix  = apply_boundary_condition('reflecting', transition_matrix, p)

initial_parameters = [num_of_steps, num_of_records, num_of_states,
                     transition_matrix, initial_distribution_matrix]

if architecture_check_passed:
    matrix_multiplication_evolution_tpdf = matrix_multiplication_evolution(initial_parameters)
    print_conclusions(num_of_steps, num_of_records, num_of_samples, matrix_multiplication_evolution_tpdf, 'matrix_multiplication_evolution')

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

array([[ 0.        ,  0.30769231,  0.        ,  0.69230769,  0.        ],
       [ 0.12307692,  0.        ,  0.46153846,  0.        ,  0.41538462],
       [ 0.        ,  0.30769231,  0.        ,  0.69230769,  0.        ],
       [ 0.12307692,  0.        ,  0.46153846,  0.        ,  0.41538462],
       [ 0.        ,  0.30769231,  0.        ,  0.69230769,  0.        ],
       [ 0.12307692,  0.        ,  0.46153846,  0.        ,  0.41538462],
       [ 0.        ,  0.30769231,  0.        ,  0.69230769,  0.        ],
       [ 0.12307692,  0.        ,  0.46153846,  0.        ,  0.41538462],
       [ 0.        ,  0.30769231,  0.        ,  0.69230769,  0.        ],
       [ 0.12307692,  0.        ,  0.46153846,  0.        ,  0.41538462]])

<IPython.core.display.Latex object>

** Semi-Reflecting Test **

In [16]:
num_of_samples     = None

transition_matrix  = create_transition_matrix(p, num_of_states)
transition_matrix  = apply_boundary_condition('semi-reflecting', transition_matrix, p)

initial_parameters = [num_of_steps, num_of_records, num_of_states,
                     transition_matrix, initial_distribution_matrix]

if architecture_check_passed:
    matrix_multiplication_evolution_tpdf = matrix_multiplication_evolution(initial_parameters)
    print_conclusions(num_of_steps, num_of_records, num_of_samples, matrix_multiplication_evolution_tpdf, 'matrix_multiplication_evolution')

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

array([[ 0.07582938,  0.11374408,  0.17061611,  0.25592417,  0.38388626],
       [ 0.07582938,  0.11374408,  0.17061611,  0.25592417,  0.38388626],
       [ 0.07582938,  0.11374408,  0.17061611,  0.25592417,  0.38388626],
       [ 0.07582938,  0.11374408,  0.17061611,  0.25592417,  0.38388626],
       [ 0.07582938,  0.11374408,  0.17061611,  0.25592417,  0.38388626],
       [ 0.07582938,  0.11374408,  0.17061611,  0.25592417,  0.38388626],
       [ 0.07582938,  0.11374408,  0.17061611,  0.25592417,  0.38388626],
       [ 0.07582938,  0.11374408,  0.17061611,  0.25592417,  0.38388626],
       [ 0.07582938,  0.11374408,  0.17061611,  0.25592417,  0.38388626],
       [ 0.07582938,  0.11374408,  0.17061611,  0.25592417,  0.38388626]])

<IPython.core.display.Latex object>

### Testing Eigen Future

** Absorbing Test **

In [17]:
num_of_steps      = None
num_of_records    = None
num_of_samples    = None

transition_matrix = create_transition_matrix(p, num_of_states)
transition_matrix = apply_boundary_condition('absorbing', transition_matrix, p)

if architecture_check_passed:
    left_eigenvectors = eigen_future(transition_matrix, initial_distribution_matrix)
    print_conclusions(num_of_steps, num_of_records, num_of_samples, left_eigenvectors, 'eigen_future')

<IPython.core.display.Latex object>

array([[ 0.30769231,  0.        ,  0.        ,  0.        ,  0.69230769],
       [ 0.30769231,  0.        ,  0.        ,  0.        ,  0.69230769]])

<IPython.core.display.Latex object>

** Reflecting Test **

In [18]:
num_of_steps      = None
num_of_records    = None
num_of_samples    = None

transition_matrix = create_transition_matrix(p, num_of_states)
transition_matrix = apply_boundary_condition('reflecting', transition_matrix, p)

if architecture_check_passed:
    left_eigenvectors = eigen_future(transition_matrix, initial_distribution_matrix)
    print_conclusions(num_of_steps, num_of_records, num_of_samples, left_eigenvectors, 'eigen_future')

<IPython.core.display.Latex object>

array([[ -1.52655666e-16,   3.07692308e-01,  -7.49400542e-16,
          6.92307692e-01,  -2.77555756e-17],
       [  1.23076923e-01,  -5.82867088e-16,   4.61538462e-01,
         -7.77156117e-16,   4.15384615e-01]])

<IPython.core.display.Latex object>

** Semi-Reflecting Test **

In [19]:
num_of_steps      = None
num_of_records    = None
num_of_samples    = None

transition_matrix = create_transition_matrix(p, num_of_states)
transition_matrix = apply_boundary_condition('semi-reflecting', transition_matrix, p)

if architecture_check_passed:
    left_eigenvectors = eigen_future(transition_matrix, initial_distribution_matrix)
    print_conclusions(num_of_steps, num_of_records, num_of_samples, left_eigenvectors, 'eigen_future')

<IPython.core.display.Latex object>

array([[ 0.07582938,  0.11374408,  0.17061611,  0.25592417,  0.38388626]])

<IPython.core.display.Latex object>

## Conclusions and Interpretations

It appears that using any method with a chosen boundary condition will yield approximately similar results. In regards to the boundary conditions, semi-reflecting produced a nice and stable distribution that was spread out across all states. Reflecting would oscillate between two distributions, and the absorbing condition forced all samples to be stuck at the boundaries.